# December 2018: Advent of Code

## Days 1-5

### Common imports & library functions

In [3]:
from collections import Counter, defaultdict, namedtuple
import doctest
import heapq
import itertools
import math
import numpy as np
import re

### Day 1: Chronal Calibration

In [43]:
def resulting_frequency(changes):
    """
    >>> resulting_frequency([+1, -2, +3, +1])
    3
    >>> resulting_frequency([+1, +1, +1])
    3
    >>> resulting_frequency([-1, -1, 2])
    0
    >>> resulting_frequency([-1, -2, -3])
    -6
    """
    return sum(changes)

def accumulate(stream):
    """
    Yields accumulated sums of elements of the input stream as an output stream. 
    The nth element of the stream is the sum of the first n elements of the input
    stream.
    
    >>> list(accumulate([0, 0, 0]))
    [0, 0, 0]
    >>> list(accumulate([1, 2, 3]))
    [1, 3, 6]
    >>> list(accumulate([-1, -2, 3]))
    [-1, -3, 0]
    """
    sum = 0
    for c in stream:
        sum += c
        yield sum

def repeated_frequency(changes):
    """
    >>> repeated_frequency([+1, -1])
    0
    >>> repeated_frequency([+3, +3, +4, -2, -4])
    10
    >>> repeated_frequency([-6, +3, +8, +5, -6])
    5
    >>> repeated_frequency([+7, +7, -2, -7, -4])
    14
    """
    seen = {0}
    for freq in accumulate(itertools.cycle(changes)):
        if freq in seen:
            return freq
        seen.add(freq)

In [44]:
# Run unit tests
doctest.testmod()

TestResults(failed=0, attempted=11)

In [45]:
# Final answers
with open('day1_input.txt') as f:
    changes = [int(l.strip()) for l in f]
    print('Part 1: ', resulting_frequency(changes))
    print('Part 2: ', repeated_frequency(changes))

Part 1:  470
Part 2:  790


### Day 2: Inventory Management System

In [36]:
def check_counts(box_id):
    """
    >>> check_counts('abcdef')
    (0, 0)
    >>> check_counts('bababc')
    (1, 1)
    >>> check_counts('aabcdd')
    (1, 0)
    """
    checks = Counter(Counter(box_id).values())
    return (1 if 2 in checks else 0, 
            1 if 3 in checks else 0)

def checksum_box_ids(box_ids):
    """
    >>> checksum_box_ids(['abcdef', 'bababc', 'abbcde', 'abcccd', 'aabcdd', 'abcdee', 'ababab'])
    12
    """
    twos, threes = np.sum([check_counts(id) for id in box_ids], axis=0)
    return twos * threes

def intersection(s, t):
    """
    >>> intersection('abcde', 'fghij')
    ''
    >>> intersection('fguij', 'fghij')
    'fgij'
    >>> intersection('abcde', 'abcde')
    'abcde'
    """
    return ''.join(s[i] for i in range(len(s)) if s[i] == t[i])

def correct_box_ids(box_ids):
    """
    >>> correct_box_ids(['abcde', 'fghij', 'klmno', 'pqrst', 'fguij', 'axcye', 'wvxyz'])
    'fgij'
    """
    box_ids = sorted(set(box_ids))
    for (i, j) in itertools.product(box_ids, box_ids):
        if i == j: continue
        solution = intersection(i, j)
        if len(solution) == len(i) - 1:
            return solution

In [37]:
# Run unit tests
doctest.testmod()

TestResults(failed=0, attempted=8)

In [40]:
# Final answers
with open('day2_input.txt') as f:
    box_ids = [l.strip() for l in f]
    print('Part 1: ', checksum_box_ids(box_ids))
    print('Part 2: ', correct_box_ids(box_ids))

Part 1:  6422
Part 2:  qcslyvphgkrmdawljuefotxbh


### Day 3: No Matter How You Slice It

In [36]:
Claim = namedtuple('Claim', ['id', 'l', 't', 'w', 'h'])

def create_fabric(w, h):
    return np.zeros((h, w), dtype=np.int)

def apply_claim(fabric, claim):
    fabric[claim.t:claim.t+claim.h, 
           claim.l:claim.l+claim.w] += 1

def apply_claims(fabric, claims):
    for claim in claims:
        apply_claim(fabric, claim)
    
def claims_in_region(fabric, r):
    return (fabric[r.t:r.t+r.h, r.l:r.l+r.w]).max()

def parse_claim(claim_id):
    """
    >>> parse_claim('#1 @ 912,277: 27x20')
    Claim(id=1, l=912, t=277, w=27, h=20)
    """
    components = [int(c) for c in re.findall('(\d+)', claim_id)]
    return Claim(*components)

def overlapping_claim_area(fabric, claims):
    """
    >>> fabric = create_fabric(8, 8)
    >>> claims = [Claim(1, 1, 3, 4, 4), Claim(2, 3, 1, 4, 4), Claim(3, 5, 5, 2, 2)]
    >>> apply_claims(fabric, claims)
    >>> overlapping_claim_area(fabric, claims)
    4
    """
    return (fabric > 1).sum()

def find_nonoverlapping_claim(fabric, claims):
    """
    >>> fabric = create_fabric(8, 8)
    >>> claims = [Claim(1, 1, 3, 4, 4), Claim(2, 3, 1, 4, 4), Claim(3, 5, 5, 2, 2)]
    >>> apply_claims(fabric, claims)
    >>> find_nonoverlapping_claim(fabric, claims)
    Claim(id=3, l=5, t=5, w=2, h=2)
    """
    for claim in claims:
        if claims_in_region(fabric, claim) == 1:
            return claim

In [37]:
# Run unit tests
doctest.testmod()

TestResults(failed=0, attempted=9)

In [38]:
# Final answers
with open('day3_input.txt') as f:
    fabric = create_fabric(1000, 1000)
    claims = [parse_claim(l.strip()) for l in f]
    apply_claims(fabric, claims)
    print('Part 1: ', overlapping_claim_area(fabric, claims))
    print('Part 2: ', find_nonoverlapping_claim(fabric, claims))

Part 1:  110891
Part 2:  Claim(id=297, l=622, t=641, w=11, h=24)
